In [1]:
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget
import numpy as np
# import tensorflow as tf
from scipy.spatial.transform import Rotation as R

import numpy as np
from matplotlib import pyplot as plt


# Draw frames from odometry constraints

In [36]:
#load relative poses from odometry file
odom_raw = np.loadtxt("pose_data.csv", delimiter=',',skiprows=1)
# print(odom_raw[0])
print(len(odom_raw))

#convert to rotation matrix
odom = np.zeros([len(odom_raw),4,4])
odom[0] = np.eye(4)
odom[:,:3,-1] = odom_raw[:,2:5]
odom[:,2,-1] *= -1 #IDK why but for some reason z is getting flipped...
for i in range(1,len(odom_raw)):
    odom[i,3,3] = 1
    odom[i,:3,:3] = R.from_quat(odom_raw[i,5:]).as_dcm() @ odom[i-1,:3,:3]
    odom[i,:3,-1] = odom[i-1,:3,-1] + odom[i,:3,-1]

65


In [ ]:
plt = Plotter(N = 1, axes = 0, bg = (0.2, 0.2, 0.2), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]         
scale = 1/10
runlen = len(odom) - 1

for i in range(1,runlen):
    rotm = odom[i,:,:]
    alph = 1-(i/runlen)
    
    #forward view direction (-z in NeRF c2w convention)
#     headings = rotm[:3,:3] @ np.array([0,0,scale])
#     disp.append(Arrows(rotm[:3,-1][None,:], (rotm[:3,-1] + headings)[None,:], c = "yellow", alpha = alph))
    # x
    headings = rotm[:3,:3] @ np.array([scale,0,0])
    disp.append(Arrows(rotm[:3,-1][None,:], (rotm[:3,-1] + headings)[None,:], c = "red", alpha = alph))
    #y
    headings = rotm[:3,:3] @ np.array([0,scale,0])
    disp.append(Arrows(rotm[:3,-1][None,:], (rotm[:3,-1] + headings)[None,:], c = "green", alpha = alph))
    #z
    headings = rotm[:3,:3] @ np.array([0,0,scale])
    disp.append(Arrows(rotm[:3,-1][None,:], (rotm[:3,-1] + headings)[None,:], c = "blue", alpha = alph))
    
    disp.append(Line(odom[i,:3,3], odom[i+1,:3,3], c = 'gray', lw=2))

plt.show(disp, "dead reckoning from odometry constraints")

# Draw point clouds

In [31]:
#load keyframe point clouds
idx = 50
fn1 = "keyframe_" + str(idx) + ".csv"
pc1 = np.loadtxt(fn1, delimiter=",")

idx2 = idx + 1
fn2 = "keyframe_" + str(idx2) + ".csv"
pc2 = np.loadtxt(fn2, delimiter=",")

In [32]:
plt = Plotter(N = 1, axes = 0, bg = (0.2, 0.2, 0.2), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]          
disp.append(Points(pc1, c = 'red', r = 3, alpha = 0.5))
disp.append(Points(pc2, c = 'blue', r = 3, alpha = 0.5))

#draw pc2 in pc1 frame 
p2_p1f = np.append(pc2, np.ones([len(pc2),1]), axis = 1)
p2_p1f = p2_p1f @ np.linalg.pinv(odom[idx]) @ odom[idx+1]
p2_p1f = p2_p1f[:,:3]
# print(p2_p1f)
disp.append(Points(p2_p1f, c = 'green', r = 3, alpha = 0.5))

plt.show(disp, "scan 1 (red), scan2 (blue), scan2 in s1 frame (green)")